## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-12-02-21-10 +0000,nypost,Tourist stabbed inside Macy’s Herald Square by...,https://nypost.com/2025/12/11/us-news/tourist-...
1,2025-12-12-02-18-00 +0000,wsj,Dollar Likely to Weaken Versus Yen in 2026,https://www.wsj.com/finance/currencies/asian-c...
2,2025-12-12-02-15-43 +0000,nypost,Wild street brawl erupts during ritzy Nantucke...,https://nypost.com/2025/12/11/us-news/wild-str...
3,2025-12-12-02-08-09 +0000,startribune,Gophers volleyball swept by Pittsburgh in NCAA...,https://www.startribune.com/gophers-volleyball...
4,2025-12-12-02-07-12 +0000,nypost,Road rage maniac busted weeks after stabbing N...,https://nypost.com/2025/12/11/us-news/road-rag...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
151,trump,42
95,oil,19
158,new,17
50,ai,15
438,venezuela,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2025-12-12-01-33-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...,105
154,2025-12-11-20-14-51 +0000,wapo,U.S. seizes ‘very large’ oil tanker off Venezu...,https://www.washingtonpost.com/world/2025/12/1...,104
17,2025-12-12-01-46-00 +0000,wsj,The Indiana Senate has rejected a redistrictin...,https://www.wsj.com/politics/policy/indiana-re...,104
335,2025-12-11-06-52-47 +0000,nyt,Trump Says U.S. Seized Oil Tanker Off Venezuel...,https://www.nytimes.com/2025/12/10/us/politics...,97
346,2025-12-11-04-00-00 +0000,wsj,President Trump wants lower interest rates and...,https://www.wsj.com/economy/central-banking/fe...,95


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,105,2025-12-12-01-33-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...
360,72,2025-12-11-02-48-00 +0000,wsj,U.S. Seizes Oil Tanker Off Venezuela in Escala...,https://www.wsj.com/world/americas/u-s-seizes-...
206,57,2025-12-11-17-27-39 +0000,nypost,Disney to let users make AI videos featuring b...,https://nypost.com/2025/12/11/business/disney-...
199,52,2025-12-11-17-51-00 +0000,wsj,The U.S. trade deficit narrowed to its lowest ...,https://www.wsj.com/economy/trade/u-s-trade-de...
17,52,2025-12-12-01-46-00 +0000,wsj,The Indiana Senate has rejected a redistrictin...,https://www.wsj.com/politics/policy/indiana-re...
85,51,2025-12-11-22-59-25 +0000,nyt,Nobel Peace Prize Winner Machado Says U.S. Hel...,https://www.nytimes.com/2025/12/11/world/ameri...
211,44,2025-12-11-17-00-00 +0000,wsj,Two progressives competing in the Democratic p...,https://www.wsj.com/politics/elections/a-bible...
55,43,2025-12-12-00-02-02 +0000,bbc,Reddit launches High Court challenge to Austra...,https://www.bbc.com/news/articles/clykk2yrl9ko...
282,43,2025-12-11-11-48-15 +0000,nypost,Heart-stopping moment skydiver’s parachute get...,https://nypost.com/2025/12/11/world-news/heart...
290,42,2025-12-11-11-00-03 +0000,nypost,ChatGPT accused of being complicit in murder f...,https://nypost.com/2025/12/11/tech/chatgpt-acc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
